# text mining (nlp) with python

**Author:** Ties de Kok ([Personal Website](http://www.tiesdekok.com))  
**Last updated:** 24 Oct 2017  
**Python version:** Python 3.5  
**License:** MIT License  

**Note:** Some features (like the ToC) will only work if you run it locally, use Binder, or use nbviewer by clicking this link: 

# *Introduction*

This notebook contains code examples to get you started with Natural Language Processing (NLP) / Text Mining for Research and Data Science purposes.  

In the large scheme of things there are roughly 4 steps:  

1. Identify a data source  
2. Gather the data  
3. Process the data  
4. Analyze the data  

This notebook only discusses step 3 and 4. If you want to learn more about step 2 see my [Python tutorial](https://github.com/TiesdeKok/LearnPythonforResearch). 

# *Elements / topics that are discussed in this notebook: *


<img style="float: left" src="https://i.imgur.com/c3aCZLA.png" width="40%" /> 

# *Table of Contents*  <a id='toc'></a>

* [Primer on NLP tools](#tool_primer)     
* [Process + Clean text](#proc_clean)   
    * [Normalization](#normalization)
        * [Deal with unwanted characters](#unwanted_char)
        * [Sentence segmentation](#sentence_seg)   
        * [Word tokenization](#word_token)
        * [Lemmatization & Stemming](#lem_and_stem) 
    * [Language modeling](#lang_model) 
        * [Part-of-Speech tagging](#pos_tagging) 
        * [Uni-Gram & N-Grams](#n_grams) 
        * [Stop words](#stop_words) 
* [Direct feature extraction](#feature_extract) 
    * [Feature search](#feature_search) 
        * [Entity recognition](#entity_recognition) 
        * [Pattern search](#pattern_search) 
    * [Text evaluation](#text_eval) 
        * [Language](#language) 
        * [Dictionary counting](#dict_counting) 
        * [Readability](#readability) 
* [Represent text numerically](#text_numerical) 
    * [Bag of Words](#bows) 
        * [TF-IDF](#tfidf) 
    * [Word Embeddings](#word_embed) 
        * [Word2Vec](#Word2Vec) 
* [Statistical models](#stat_models) 
    * ["Traditional" machine learning](#trad_ml) 
        * [Supervised](#trad_ml_supervised) 
            * [Naïve Bayes](#trad_ml_supervised_nb) 
            * [Support Vector Machines (SVM)](#trad_ml_supervised_svm) 
        * [Unsupervised](#trad_ml_unsupervised) 
            * [Latent Dirichilet Allocation (LDA)](#trad_ml_unsupervised_lda) 
            * [pyLDAvis](#trad_ml_unsupervised_pyLDAvis) 
* [Model Selection and Evaluation](#trad_ml_eval) 
* [Neural Networks](#nn_ml)

# <span style="text-decoration: underline;">Primer on NLP tools</span><a id='tool_primer'></a> [(to top)](#toc)

There are many tools available for NLP purposes.  
The code examples below are based on what I personally like to use, it is not intended to be a comprehsnive overview.  

Besides build-in Python functionality I will use / demonstrate the following packages:

**Standard NLP libraries**:
1. `Spacy` and the higher-level wrapper `Textacy` 
2. `NLTK` and the higher-level wrapper `TextBlob`

*Note: besides installing the above packages you also often have to download (model) data . Make sure to check the documentation!*

**Standard machine learning library**:

1. `scikit learn`

**Topic modelling libraries**:

1. `Gensim` 
2. `FastText`

**Specific task libraries**:

There are many, just a couple of examples:

1. `pyLDAvis` for visualizing LDA)
2. `langdetect` for detecting languages
3. `fuzzywuzzy` for fuzzy text matching
4. `textstat` to calculate readability statistics

** Neural Network (+ Deep Learning) libraries**:

1. `Tensorflow`  
2. `PyTorch`  
3. `Keras`

# <span style="text-decoration: underline;">Get some example data</span><a id='example_data'></a> [(to top)](#toc)

There are many example datasets available to play around with, see for example this great repository:  
https://archive.ics.uci.edu/ml/datasets.html?format=&task=&att=&area=&numAtt=&numIns=&type=text&sort=nameUp&view=table

The data that I will use for most of the examples is the "Reuter_50_50 Data Set" that is used for author identification experiments. 

See the details here: https://archive.ics.uci.edu/ml/datasets/Reuter_50_50  

### Download and load the data

Can't follow what I am doing here? Please see my [Python tutorial](https://github.com/TiesdeKok/LearnPythonforResearch).

In [1]:
import requests, zipfile, io, os

*Download and extract the zip file with the data *

In [2]:
if not os.path.exists('C50test'):
    r = requests.get("https://archive.ics.uci.edu/ml/machine-learning-databases/00217/C50.zip")
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall()

*Load the data into memory*

In [3]:
folder_dict = {'test' : 'C50test', 'train' : 'C50train'}
text_dict = {'test' : {}, 'train' : {}}

In [4]:
for label, folder in folder_dict.items():
    authors = os.listdir(folder)
    for author in authors:
        text_files = os.listdir(os.path.join(folder, author))
        for file in text_files:
            with open(os.path.join(folder, author, file), 'r') as text_file:
                text_dict[label].setdefault(author, []).append(' '.join(text_file.readlines()))

*Note: the text comes pre-split per sentence, for the sake of example I undo this doing `' '.join(text_file.readlines()`*

In [5]:
text_dict['test']['TimFarrand'][0]

'Shares in brewing-to-leisure group Bass Plc are likely to be held back until Britain\'s Trade and Industry secretary Ian Lang decides whether to allow its proposed merge with brewer Carlsberg-Tetley, said analysts.\n Earlier Lang announced the Bass deal would be referred to the Monoplies and Mergers Commission which is due to report before March 24, 1997. The shares fell 6p to 781p on the news.\n "The stock is probably dead in the water until March," said John Wakley, analyst at Lehman Brothers.  \n Dermott Carr, an analyst at Nikko said, "the market is going to hang onto them for the moment but until we get a decision they will be held back."\n Whatever the MMC decides many analysts expect Lang to defer a decision until after the next general election which will be called by May 22.\n "They will probably try to defer the decision until after the election. I don\'t think they want the negative PR of having a large number of people fired," said Wakley.  \n If the deal does not go throu

# <span style="text-decoration: underline;">Process + Clean text</span><a id='proc_clean'></a> [(to top)](#toc)

## Convert the text into a NLP representation

We can use the text directly, but if want to use packages like `spacy` and `nltk` / `textblob` we first have to convert the text into a corresponding object.  

### Spacy

In [6]:
from spacy.en import English
parser = English()

Convert all text in the "test" sample to a `spacy` `doc` object using `parser()`:

In [7]:
spacy_text = {}
for author, text_list in text_dict['test'].items():
    spacy_text[author] = [parser(text) for text in text_list]

In [8]:
type(spacy_text['TimFarrand'][0])

spacy.tokens.doc.Doc

### NLTK

In [9]:
import nltk

We can apply basic `nltk` operations directly to the text so we don't need to convert first. 

### TextBlob

In [10]:
from textblob import TextBlob

Convert all text in the "test" sample to a `TextBlob` object using `TextBlob()`:

In [11]:
textblob_text = {}
for author, text_list in text_dict['test'].items():
    textblob_text[author] = [TextBlob(text) for text in text_list]

In [12]:
type(textblob_text['TimFarrand'][0])

textblob.blob.TextBlob

## <span style="text-decoration: underline;">Normalization</span><a id='normalization'></a> [(to top)](#toc)

The goal of **text normalization** describes the task of transforming the text into a different form.  

This can imply many things, I will show a couple of things below:

### <span style="text-decoration: underline;">Deal with unwanted characters</span><a id='unwanted_char'></a> [(to top)](#toc)

You will often notice that there are characters that you don't want in your text.  

Let's look at this sentence for example:

> "Shares in brewing-to-leisure group Bass Plc are likely to be held back until Britain\'s Trade and Industry secretary Ian Lang decides whether to allow its proposed merge with brewer Carlsberg-Tetley, said analysts.\n Earlier Lang announced the Bass deal would be referred to the Monoplies and Mergers"

You notice that there are some `\` and `\n` in there. These are used to define how a string should be displayed, if we print this text we get:  

In [13]:
text_dict['test']['TimFarrand'][0][:298]

"Shares in brewing-to-leisure group Bass Plc are likely to be held back until Britain's Trade and Industry secretary Ian Lang decides whether to allow its proposed merge with brewer Carlsberg-Tetley, said analysts.\n Earlier Lang announced the Bass deal would be referred to the Monoplies and Mergers"

In [14]:
print(text_dict['test']['TimFarrand'][0][:298])

Shares in brewing-to-leisure group Bass Plc are likely to be held back until Britain's Trade and Industry secretary Ian Lang decides whether to allow its proposed merge with brewer Carlsberg-Tetley, said analysts.
 Earlier Lang announced the Bass deal would be referred to the Monoplies and Mergers


If we want to analyze text we often don't care about the visual representation. They might actually cause problems!  

** So how do we remove them? **

In many cases it is sufficient to simply use the `.replace()` function:

In [15]:
text_dict['test']['TimFarrand'][0][:298].replace('\n', '').replace('\\', '')

"Shares in brewing-to-leisure group Bass Plc are likely to be held back until Britain's Trade and Industry secretary Ian Lang decides whether to allow its proposed merge with brewer Carlsberg-Tetley, said analysts. Earlier Lang announced the Bass deal would be referred to the Monoplies and Mergers"

Sometimes, however, the problem arrises because of encoding / decoding problems.  

In those cases you can usually do something like:  

In [16]:
problem_sentence = 'This is some \\u03c0 text that has to be cleaned\\u2026! it\\u0027s annoying!'
print(problem_sentence.encode().decode('unicode_escape').encode('ascii','ignore'))

b"This is some  text that has to be cleaned! it's annoying!"


### <span style="text-decoration: underline;">Sentence segmentation</span><a id='sentence_seg'></a> [(to top)](#toc)

Sentence segmentation means the task of splitting up the piece of text by sentence.  

You could do this by splitting on the `.` symbol, but dots are used in many other cases as well so it is not very robust:

In [17]:
text_dict['test']['TimFarrand'][0][:550].split('.')

["Shares in brewing-to-leisure group Bass Plc are likely to be held back until Britain's Trade and Industry secretary Ian Lang decides whether to allow its proposed merge with brewer Carlsberg-Tetley, said analysts",
 '\n Earlier Lang announced the Bass deal would be referred to the Monoplies and Mergers Commission which is due to report before March 24, 1997',
 ' The shares fell 6p to 781p on the news',
 '\n "The stock is probably dead in the water until March," said John Wakley, analyst at Lehman Brothers',
 '  \n Dermott Carr, an analyst at Nikko said, "the mark']

It is better to use a more sophisticated implementation such as the one by `Spacy`:

In [18]:
example_paragraph = spacy_text['TimFarrand'][0]

In [19]:
sentence_list = [s for s in example_paragraph.sents]
sentence_list[:5]

[Shares in brewing-to-leisure group Bass Plc are likely to be held back until Britain's Trade and Industry secretary Ian Lang decides whether to allow its proposed merge with brewer Carlsberg-Tetley, said analysts.
  ,
 Earlier Lang announced the Bass deal would be referred to the Monoplies and Mergers Commission which is due to report before March 24, 1997.,
 The shares fell 6p to 781p on the news.
  ,
 "The stock is probably dead in the water until March," said John Wakley, analyst at Lehman Brothers.  
  ,
 Dermott Carr, an analyst at Nikko said, "the market is going to hang onto them for the moment but until we get a decision they will be held back."
  ]

Notice that the returned object is still a `spacy` object:

In [20]:
type(sentence_list[0])

spacy.tokens.span.Span

Apply to all texts (for use later on):

In [21]:
spacy_sentences = {}
for author, text_list in spacy_text.items():
    spacy_sentences[author] = [[s for s in text.sents] for text in text_list]

In [22]:
spacy_sentences['TimFarrand'][0][:3]

[Shares in brewing-to-leisure group Bass Plc are likely to be held back until Britain's Trade and Industry secretary Ian Lang decides whether to allow its proposed merge with brewer Carlsberg-Tetley, said analysts.
  ,
 Earlier Lang announced the Bass deal would be referred to the Monoplies and Mergers Commission which is due to report before March 24, 1997.,
 The shares fell 6p to 781p on the news.
  ]

### <span style="text-decoration: underline;">Word tokenization</span><a id='word_token'></a> [(to top)](#toc)

Word tokenization means to split the sentence (or text) up into words.

In [23]:
example_sentence = spacy_sentences['TimFarrand'][0][0]
example_sentence

Shares in brewing-to-leisure group Bass Plc are likely to be held back until Britain's Trade and Industry secretary Ian Lang decides whether to allow its proposed merge with brewer Carlsberg-Tetley, said analysts.
 

A word is called a `token` in this context (hence `tokenization`), using `spacy`:

In [24]:
token_list = [token for token in example_sentence]
token_list[0:15]

[Shares,
 in,
 brewing,
 -,
 to,
 -,
 leisure,
 group,
 Bass,
 Plc,
 are,
 likely,
 to,
 be,
 held]

### <span style="text-decoration: underline;">Lemmatization & Stemming</span><a id='lem_and_stem'></a> [(to top)](#toc)

In some cases you want to convert a word (i.e. token) into a more general representation.  

For example: convert "car", "cars", "car's", "cars'" all into the word `car`.

This is generally done through lemmatization / stemming (different approaches trying to achieve a similar goal).  

**Spacy**

Space offers build-in functionality for lemmatization:

In [25]:
lemmatized = [token.lemma_ for token in example_sentence]
lemmatized[0:15]

['share',
 'in',
 'brewing',
 '-',
 'to',
 '-',
 'leisure',
 'group',
 'bass',
 'plc',
 'be',
 'likely',
 'to',
 'be',
 'hold']

**NLTK**

Using the NLTK libary we can also use the more aggressive Porter Stemmer

In [26]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [27]:
stemmed = [stemmer.stem(token.text) for token in example_sentence]
stemmed[0:15]

['share',
 'in',
 'brew',
 '-',
 'to',
 '-',
 'leisur',
 'group',
 'bass',
 'plc',
 'are',
 'like',
 'to',
 'be',
 'held']

**Compare**:

In [28]:
for original, lemma, stem in zip(token_list[:15], lemmatized[:15], stemmed[:15]):
    print(original, ' | ', lemma, ' | ', stem)

Shares  |  share  |  share
in  |  in  |  in
brewing  |  brewing  |  brew
-  |  -  |  -
to  |  to  |  to
-  |  -  |  -
leisure  |  leisure  |  leisur
group  |  group  |  group
Bass  |  bass  |  bass
Plc  |  plc  |  plc
are  |  be  |  are
likely  |  likely  |  like
to  |  to  |  to
be  |  be  |  be
held  |  hold  |  held


As you can see, the Porter Stemmer is the most aggressive. In my experience it is usually best to use lemmatization instead of a stemmer. 

## <span style="text-decoration: underline;">Language modeling</span><a id='lang_model'></a> [(to top)](#toc)

Text is inherently structured in complex ways, we can often use some of this underlying structure. 

### <span style="text-decoration: underline;">Part-of-Speech tagging</span><a id='pos_tagging'></a> [(to top)](#toc)

Part of speech tagging refers to the identification of words as nouns, verbs, adjectives, etc. 

Using `Spacy`:

In [29]:
pos_list = [(token, token.pos_) for token in example_sentence]
pos_list[0:10]

[(Shares, 'NOUN'),
 (in, 'ADP'),
 (brewing, 'NOUN'),
 (-, 'PUNCT'),
 (to, 'ADP'),
 (-, 'PUNCT'),
 (leisure, 'NOUN'),
 (group, 'NOUN'),
 (Bass, 'PROPN'),
 (Plc, 'PROPN')]

### <span style="text-decoration: underline;">Uni-Gram & N-Grams</span><a id='n_grams'></a> [(to top)](#toc)

Obviously a sentence is not a random collection of words, the sequence of words has information value.  

A simple way to incorporate some of this sequence is by using what is called `n-grams`.  
An `n-gram` is nothing more than a a combination of `N` words into one "word" (a uni-gram is just one word).  

So we can convert `"Sentence about flying cars"` into a list of bigrams:

> Sentence-about, about-flying, flying-cars

Using `NLTK`:

In [30]:
bigram_list = ['-'.join(x) for x in nltk.bigrams([token.text for token in example_sentence])]
bigram_list[10:15]

['are-likely', 'likely-to', 'to-be', 'be-held', 'held-back']

### <span style="text-decoration: underline;">Stop words</span><a id='stop_words'></a> [(to top)](#toc)

Depending on what you are trying to do it is possible that there are many words that don't add any information value to the sentence.  

The primary example are stop words.  

Sometimes you can improve the accuracy of your model by removing stop words.

Using `Spacy`:

In [31]:
no_stop_words = [token for token in example_sentence if not token.is_stop]

In [32]:
no_stop_words[:10]

[Shares, brewing, -, -, leisure, group, Bass, Plc, likely, held]

In [33]:
token_list[:10]

[Shares, in, brewing, -, to, -, leisure, group, Bass, Plc]

## Use the awesome `textacy` package to save time

`Textacy` is a high-level wrapper built on `spaCy`, it has many cool features!

See their GitHub page for details: https://github.com/chartbeat-labs/textacy

In [34]:
import textacy

### We can almost completely wrap all of our pre-processing into one `textacy` command:

http://textacy.readthedocs.io/en/latest/api_reference.html#module-textacy.preprocess

In [35]:
example_text = text_dict['test']['TimFarrand'][0]

In [36]:
cleaned_text = textacy.preprocess_text(example_text, lowercase=True, fix_unicode=True, no_punct=True)

** Let's use the above + textacy to do some basic preprocessing **

1. Split into sentences
2. Apply lemmatizer
3. Clean up the sentence using `textacy`

Note: using textacy can be quite slow so use with caution! 

In [37]:
spacy_text_clean = {}
for author, text_list in text_dict['test'].items():
    lst = []
    for text in text_list:
        sentences = [' '.join([token.lemma_ for token in s]) for s in parser(text).sents]
        sentences_cleaned = [textacy.preprocess_text(sen, lowercase=True, fix_unicode=True, no_punct=True) for sen in sentences]
        lst.append([parser(sen) for sen in sentences_cleaned])
    spacy_text_clean[author] = lst

In [38]:
spacy_text_clean['TimFarrand'][0][:3]

[share in brewing to leisure group bass plc be likely to be hold back until britain s trade and industry secretary ian lang decide whether to allow pron propose merge with brewer carlsberg tetley say analyst,
 earlier lang announce the bass deal would be refer to the monoplies and mergers commission which be due to report before march 24 1997,
 the share fall 6p to 781p on the news]

# <span style="text-decoration: underline;">Direct feature extraction</span><a id='feature_extract'></a> [(to top)](#toc)

We now have pre-processed our text into something that we can use.  

The easiest thing we can do now is what I label "direct feature extraction"

## <span style="text-decoration: underline;">Feature search</span><a id='feature_search'></a> [(to top)](#toc)

There are many features that you can extract from the text.

### <span style="text-decoration: underline;">Entity recognition</span><a id='entity_recognition'></a> [(to top)](#toc)

It is often useful / relevant to extract entities that are mentioned in a piece of text. 

Example using `textacy`

In [39]:
example_sentence = spacy_text_clean['TimFarrand'][0][8]
example_sentence

if the deal do not go through analyst calculate the maximum loss to bass of 60 million with most sum centre on the 30 40 million range

Extract numeric entities:

In [40]:
for sen in spacy_text_clean['TimFarrand'][4]:
    print([(token.text, token.ent_type_) for token in sen if token.ent_type_ != ''])

[('a', 'PERCENT'), ('48', 'PERCENT'), ('percent', 'PERCENT'), ('1487', 'CARDINAL'), ('million', 'CARDINAL'), ('2464', 'CARDINAL'), ('million', 'CARDINAL'), ('november', 'DATE'), ('1995', 'DATE')]
[('the', 'DATE'), ('year', 'DATE')]
[('the', 'DATE'), ('current', 'DATE'), ('year', 'DATE')]
[('the', 'DATE'), ('year', 'DATE'), ('end', 'DATE'), ('27', 'CARDINAL'), ('1996', 'DATE'), ('312', 'CARDINAL'), ('million', 'CARDINAL'), ('239', 'CARDINAL'), ('million', 'CARDINAL')]
[('107', 'PERCENT'), ('percent', 'PERCENT'), ('1113', 'CARDINAL'), ('million', 'CARDINAL')]
[('471', 'MONEY'), ('million', 'MONEY'), ('the', 'DATE'), ('11', 'DATE'), ('month', 'DATE'), ('18', 'MONEY'), ('million', 'MONEY')]
[('145', 'CARDINAL'), ('150', 'CARDINAL'), ('million', 'CARDINAL')]
[('597p', 'CARDINAL'), ('pron', 'ORG')]
[('one', 'CARDINAL')]
[('1997', 'DATE')]
[]
[]
[('britain', 'GPE'), ('annual', 'DATE'), ('december', 'DATE')]
[('170', 'CARDINAL'), ('million', 'CARDINAL'), ('199798', 'DATE'), ('about', 'CARDINAL

### <span style="text-decoration: underline;">Pattern search</span><a id='pattern_search'></a> [(to top)](#toc)

Using the build-in `re` (regular expression) library you can pattern match anything you want.  

I will not go into details about regular expressions but see here for a tutorial:  
https://regexone.com/references/python  

In [41]:
import re

**TIP**: Use [Pythex.org](https://pythex.org/) to try out your regular expression

Example on Pythex: <a href="https://pythex.org/?regex=IDNUMBER: (\d\d\d-\w\w)&test_string=Ties de Kok (IDNUMBER: 123-AZ). Rest of Text." target='_blank'>click here</a>

**Example 1:**  

In [42]:
string_1 = 'Ties de Kok (#IDNUMBER: 123-AZ). Rest of text...'
string_2 = 'Philip Joos (#IDNUMBER: 663-BY). Rest of text...'

In [43]:
pattern = r'#IDNUMBER: (\d\d\d-\w\w)'

In [44]:
print(re.findall(pattern, string_1)[0])
print(re.findall(pattern, string_2)[0])

123-AZ
663-BY


### Example 2:

If a sentence contains the word 'million' return True, otherwise return False

In [45]:
for sen in spacy_text_clean['TimFarrand'][2]:
    TERM = 'million'
    contains = True if re.search('million', sen.text) else False
    if contains:
        print(sen)

analyst forecast for pretax profit range from 218 to 232 million stg after restructure cost up from 206 million last time
a restructure cost of some 35 million be anticipate with the bulk of pron or about 25 million stem from the closure of pron small production plant in france
cadbury s us drink business should turn in about 112 million stg in trading profit against 59 million in the first half of 1995 due entirely to the contribution of dr pepper
campbell estimate uk beverage will contribute 47 million stg in operating profit down from 50 million last time
broadly analyst expect a pretty flat performance from the group s confectionery business with a consensus forecast of 110 million stg for operate profit
on average analyst calculate beverage will chip in trading profit of 150 million
after the sale of pron 51 percent stake in the coca cola amp schweppes beverages ccsb operation to coca cola enterprises in june for 620 million stg many analyst want to see a clear statement of strate

## <span style="text-decoration: underline;">Text evaluation</span><a id='text_eval'></a> [(to top)](#toc)

Besides search for features there are also many ways to analyze the text as a whole.  

Let's, for example, evaluate the following paragraph:

In [46]:
example_paragraph = ' '.join([x.text for x in spacy_text_clean['TimFarrand'][2]])
example_paragraph[:500]

'soft drink and confectionery group cadbury schweppes plc expect to report a solid eight percent rise in first half profit on wednesday face question about the performance of pron 7up soft drink in the us one of the main question will be the success or otherwise of the relaunch of the 7up brand say mark duffy food manufacturing analyst at sbc warburg competitor sprite own by coca cola have see an agressive marketing push and be rank the fast grow brand in the us after cadbury s dr pepper analyst '

### <span style="text-decoration: underline;">Language</span><a id='language'></a> [(to top)](#toc)

Using the `langdetect` package it is easy to detect the language of a piece of text

In [47]:
from langdetect import detect

In [48]:
detect(example_paragraph)

'en'

### <span style="text-decoration: underline;">Readability</span><a id='readability'></a> [(to top)](#toc)

Using the `textstat` package we can compute various readability metrics

https://github.com/shivam5992/textstat

In [49]:
from textstat.textstat import textstat

In [50]:
print(textstat.flesch_reading_ease(example_paragraph))
print(textstat.smog_index(example_paragraph))
print(textstat.flesch_kincaid_grade(example_paragraph))
print(textstat.coleman_liau_index(example_paragraph))
print(textstat.automated_readability_index(example_paragraph))
print(textstat.dale_chall_readability_score(example_paragraph))
print(textstat.difficult_words(example_paragraph))
print(textstat.linsear_write_formula(example_paragraph))
print(textstat.gunning_fog(example_paragraph))
print(textstat.text_standard(example_paragraph))

-610.94
0
269.6
10.42
344.4
40.86
133
54.0
285.3213352685051
269th and 270th grade


## Text similarity

In [51]:
from fuzzywuzzy import fuzz

In [52]:
fuzz.ratio("fuzzy wuzzy was a bear", "wuzzy fuzzy was a bear")

91

### <span style="text-decoration: underline;">Term (dictionary) counting</span><a id='dict_counting'></a> [(to top)](#toc)

One of the most common techniques that researchers currently use (at least in Accounting research) are simple metrics based on counting words in a dictionary.  
This technique is, for example, very prevalent in sentiment analysis (counting positive and negative words).  

In essence this technique is very simple to program:

### Example 1:

In [53]:
word_dictionary = ['soft', 'first', 'most', 'be']

In [54]:
for word in word_dictionary:
    print(word, example_paragraph.count(word))

soft 3
first 3
most 1
be 25


### Example 2:

In [55]:
pos = ['great', 'increase']
neg = ['bad', 'decrease']

In [56]:
sentence = '''According to Trump everything is great, great, 
and great even though his popularity is seeing a decrease.'''

In [57]:
pos_count = 0
for word in pos:
    pos_count += sentence.lower().count(word)
print(pos_count)

3


In [58]:
neg_count = 0
for word in neg:
    neg_count += sentence.lower().count(word)
print(neg_count)

1


In [59]:
pos_count / (neg_count + pos_count)

0.75

Getting the total number of words is also easy:

In [60]:
len(parser(example_paragraph))

689

#### Example 3:

We can also save the count per word

In [61]:
pos_count_dict = {}
for word in pos:
    pos_count_dict[word] = sentence.lower().count(word)

In [62]:
pos_count_dict

{'great': 3, 'increase': 0}

# <span style="text-decoration: underline;">Represent text numerically</span><a id='text_numerical'></a> [(to top)](#toc)

## <span style="text-decoration: underline;">Bag of Words</span><a id='bows'></a> [(to top)](#toc)

Sklearn includes the `CountVectorizer` and `TfidfVectorizer` function.  

For details, see the documentation:  
[TF](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer)  
[TFIDF](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer)

Note: these functions also already includes a lot of preprocessing options (e.g. ngrames, remove stop words, accent stripper)

In [63]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

### Simple example:

In [64]:
doc_1 = "The sky is blue."
doc_2 = "The sun is bright today."
doc_3 = "The sun in the sky is bright."
doc_4 = "We can see the shining sun, the bright sun."

Calculate term frequency:

In [65]:
vectorizer = CountVectorizer(stop_words='english')
tf = vectorizer.fit_transform([doc_1, doc_2, doc_3, doc_4])

In [66]:
print(vectorizer.get_feature_names())
for doc_tf_vector in tf.toarray():
    print(doc_tf_vector)

['blue', 'bright', 'shining', 'sky', 'sun', 'today']
[1 0 0 1 0 0]
[0 1 0 0 1 1]
[0 1 0 1 1 0]
[0 1 1 0 2 0]


### <span style="text-decoration: underline;">TF-IDF</span><a id='tfidf'></a> [(to top)](#toc)

In [67]:
transformer = TfidfVectorizer(stop_words='english')
tfidf = transformer.fit_transform([doc_1, doc_2, doc_3, doc_4])

In [68]:
for doc_vector in tfidf.toarray():
    print(doc_vector)

[ 0.78528828  0.          0.          0.6191303   0.          0.        ]
[ 0.          0.47380449  0.          0.          0.47380449  0.74230628]
[ 0.          0.53256952  0.          0.65782931  0.53256952  0.        ]
[ 0.          0.36626037  0.57381765  0.          0.73252075  0.        ]


### More elaborate example:

In [69]:
clean_paragraphs = []
for author, value in spacy_text_clean.items():
    for article in value:
        clean_paragraphs.append(' '.join([x.text for x in article]))

In [70]:
len(clean_paragraphs)

2500

In [71]:
transformer = TfidfVectorizer(stop_words='english')
tfidf_large = transformer.fit_transform(clean_paragraphs)

In [72]:
print('Number of vectors:', len(tfidf_large.toarray()))
print('Number of words in dictionary:', len(tfidf_large.toarray()[0]))

Number of vectors: 2500
Number of words in dictionary: 26902


In [73]:
tfidf_large

<2500x26902 sparse matrix of type '<class 'numpy.float64'>'
	with 450663 stored elements in Compressed Sparse Row format>

## <span style="text-decoration: underline;">Word Embeddings</span><a id='word_embed'></a> [(to top)](#toc)

### <span style="text-decoration: underline;">Word2Vec</span><a id='Word2Vec'></a> [(to top)](#toc)

Simple example below is from:  https://medium.com/@mishra.thedeepak/word2vec-in-minutes-gensim-nlp-python-6940f4e00980

In [74]:
import gensim
from nltk.corpus import brown

In [75]:
sentences = brown.sents()
model = gensim.models.Word2Vec(sentences, min_count=1)

Save model

In [76]:
model.save('brown_model')

Load model

In [77]:
model = gensim.models.Word2Vec.load('brown_model')

Find words most similar to 'mother':

In [78]:
print(model.most_similar("mother"))

[('father', 0.9754652976989746), ('husband', 0.9616575837135315), ('wife', 0.9492062330245972), ('son', 0.9271498918533325), ('voice', 0.9259058237075806), ('friend', 0.9201636910438538), ('boy', 0.9127996563911438), ('doctor', 0.8933248519897461), ('maid', 0.8927353620529175), ('uncle', 0.88885498046875)]


Find the odd one out:

In [79]:
print(model.doesnt_match("breakfast cereal dinner lunch".split()))

cereal


In [80]:
print(model.doesnt_match("pizza pasta garden fries".split()))

garden


Retrieve vector representation of the word "human"

In [81]:
model['human']

array([  2.82445643e-02,  -2.40943223e-01,  -6.55970454e-01,
        -4.80932444e-01,  -4.06032711e-01,   1.36476532e-01,
        -1.59991145e-01,  -2.74191685e-02,   4.31680381e-01,
         6.55596256e-01,   1.25325048e+00,  -7.73438334e-01,
        -5.25921881e-01,  -1.30807951e-01,   3.34413707e-01,
        -1.76354930e-01,   1.27241477e-01,   2.61553586e-01,
         3.84649962e-01,  -8.61963928e-01,  -3.18135411e-01,
         1.57137334e-01,  -8.10451388e-01,   7.75934875e-01,
         2.33956248e-01,  -6.89105690e-01,   1.72768995e-01,
        -5.06303966e-01,  -6.60854578e-02,   5.26066184e-01,
        -1.04654598e+00,  -8.30319002e-02,   9.10344347e-02,
         8.68004411e-02,  -9.44068611e-01,  -8.71582210e-01,
         6.06841087e-01,  -6.07468367e-01,   1.88034549e-01,
        -1.36444676e+00,  -3.32189739e-01,   4.59514678e-01,
         3.41839641e-01,   1.08876240e+00,   5.81619680e-01,
        -2.33794630e-01,   5.90388179e-01,  -1.32641211e-01,
         1.37848902e+00,

# <span style="text-decoration: underline;">Statistical models</span><a id='stat_models'></a> [(to top)](#toc)

## <span style="text-decoration: underline;">"Traditional" machine learning</span><a id='trad_ml'></a> [(to top)](#toc)

The library to use for machine learning is scikit-learn (["sklearn"](http://scikit-learn.org/stable/index.html)).

## <span>Supervised</span><a id='trad_ml_supervised'></a> [(to top)](#toc)

In [82]:
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.externals import joblib

In [83]:
import pandas as pd
import numpy as np

### Convert the data into a pandas dataframe (so that we can input it easier)

In [84]:
article_list = []
for author, value in spacy_text_clean.items():
    for article in value:
        article_list.append((author, ' '.join([x.text for x in article])))

In [85]:
article_df = pd.DataFrame(article_list, columns=['author', 'text'])

In [86]:
article_df.sample(5)

,author,text
624,AaronPressman,the chairman of an influential national resear...
1674,NickLouth,long distance telephone company and regional b...
1247,EricAuchard,att corp say thursday pron withdraw pron appli...
1994,RogerFillion,the clinton administration thursday throw pron...
680,EdnaFernandes,us car giant ford motor co say on thursday pro...


### Split the sample into a training and test sample

In [87]:
X_train, X_test, y_train, y_test = train_test_split(article_df.text, article_df.author, test_size=0.20, random_state=3561)

In [88]:
print(len(X_train), len(X_test))

2000 500


### Train and evaluate function

Simple function to train (i.e. fit) and evaluate the model

In [89]:
def train_and_evaluate(clf, X_train, X_test, y_train, y_test):
    
    clf.fit(X_train, y_train)
    
    print("Accuracy on training set:")
    print(clf.score(X_train, y_train))
    print("Accuracy on testing set:")
    print(clf.score(X_test, y_test))
    
    y_pred = clf.predict(X_test)
    
    print("Classification Report:")
    print(metrics.classification_report(y_test, y_pred))

### <span>Naïve Bayes estimator</span><a id='trad_ml_supervised_nb'></a> [(to top)](#toc)

In [90]:
from sklearn.naive_bayes import MultinomialNB

Define pipeline

In [91]:
clf = Pipeline([
    ('vect', TfidfVectorizer(strip_accents='unicode',
                             lowercase = True,
                            max_features = 1500,
                            stop_words='english'
                            )),
        
    ('clf', MultinomialNB(alpha = 1,
                          fit_prior = True
                          )
    ),
])

Train and show evaluation stats

In [92]:
train_and_evaluate(clf, X_train, X_test, y_train, y_test)

Accuracy on training set:
0.8355
Accuracy on testing set:
0.678
Classification Report:
                   precision    recall  f1-score   support

    AaronPressman       0.88      0.78      0.82         9
       AlanCrosby       0.52      1.00      0.69        11
   AlexanderSmith       1.00      0.88      0.93         8
  BenjaminKangLim       0.60      0.33      0.43         9
    BernardHickey       0.43      0.75      0.55         8
      BradDorfman       1.00      0.43      0.60        14
 DarrenSchuettler       0.71      1.00      0.83        10
      DavidLawder       0.57      0.44      0.50         9
    EdnaFernandes       1.00      0.30      0.46        10
      EricAuchard       0.33      0.22      0.27         9
   FumikoFujisaki       1.00      1.00      1.00        10
   GrahamEarnshaw       0.47      0.70      0.56        10
 HeatherScoffield       1.00      0.64      0.78        14
       JanLopatka       0.78      0.58      0.67        12
    JaneMacartney       0.6

Save results

In [93]:
joblib.dump(clf, 'naive_bayes_results.pkl')

['naive_bayes_results.pkl']

Predict out of sample:

In [94]:
example_y, example_X = y_train[33], X_train[33]

In [95]:
print('Actual author:', example_y)
print('Predicted author:', clf.predict([example_X])[0])

Actual author: TanEeLyn
Predicted author: TanEeLyn


### <span>Support Vector Machines (SVM)</span><a id='trad_ml_supervised_svm'></a> [(to top)](#toc)

In [96]:
from sklearn.svm import SVC

Define pipeline

In [97]:
clf_svm = Pipeline([
    ('vect', TfidfVectorizer(strip_accents='unicode',
                             lowercase = True,
                            max_features = 1500,
                            stop_words='english'
                            )),
        
    ('clf', SVC(kernel='rbf' ,
                C=10, gamma=0.3)
    ),
])

*Note:* The SVC estimator is very sensitive to the hyperparameters!

Train and show evaluation stats

In [98]:
train_and_evaluate(clf_svm, X_train, X_test, y_train, y_test)

Accuracy on training set:
0.997
Accuracy on testing set:
0.814
Classification Report:
                   precision    recall  f1-score   support

    AaronPressman       1.00      1.00      1.00         9
       AlanCrosby       0.77      0.91      0.83        11
   AlexanderSmith       0.78      0.88      0.82         8
  BenjaminKangLim       0.80      0.89      0.84         9
    BernardHickey       0.73      1.00      0.84         8
      BradDorfman       0.90      0.64      0.75        14
 DarrenSchuettler       0.91      1.00      0.95        10
      DavidLawder       0.67      0.67      0.67         9
    EdnaFernandes       0.86      0.60      0.71        10
      EricAuchard       0.75      0.33      0.46         9
   FumikoFujisaki       1.00      1.00      1.00        10
   GrahamEarnshaw       0.73      0.80      0.76        10
 HeatherScoffield       0.93      0.93      0.93        14
       JanLopatka       0.77      0.83      0.80        12
    JaneMacartney       0.71

Save results

In [99]:
joblib.dump(clf_svm, 'svm_results.pkl')

['svm_results.pkl']

Predict out of sample:

In [100]:
example_y, example_X = y_train[33], X_train[33]

In [101]:
print('Actual author:', example_y)
print('Predicted author:', clf_svm.predict([example_X])[0])

Actual author: TanEeLyn
Predicted author: TanEeLyn


## <span>Model Selection and Evaluation</span><a id='trad_ml_eval'></a> [(to top)](#toc)

Both the `TfidfVectorizer` and `SVC()` estimator take a lot of hyperparameters.  

It can be difficult to figure out what the best parameters are.

We can use `GridSearchCV` to help figure this out.

In [102]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score

First we define the options that should be tried out:

In [103]:
print('lala')

lala


In [104]:
print('test')

test


In [105]:
clf_search = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', SVC())
])
parameters = { 'vect__stop_words': ['english'],
                'vect__strip_accents': ['unicode'],
              'vect__max_features' : [1500],
              'vect__ngram_range': [(1,1), (2,2) ],
             'clf__gamma' : [0.2, 0.3, 0.4], 
             'clf__C' : [8, 10, 12],
              'clf__kernel' : ['rbf']
             }

Run everything:

In [106]:
grid = GridSearchCV(clf_search, param_grid=parameters, scoring=make_scorer(f1_score, average='micro'), n_jobs=6)
grid.fit(X_train, y_train)    

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))]),
       fit_params={}, iid=True, n_jobs=6,
       param_grid={'clf__gamma': [0.2, 0.3, 0.4], 'vect__max_features': [1500], 'clf__kernel': ['rbf'], 'clf__C': [8, 10, 12], 'vect__strip_accents': ['unicode'], 'vect__ngram_range': [(1, 1), (2, 2)], 'vect__stop_words': ['english']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=make_scorer(f1_score, average=micro), verbose=0)

*Note:* if you are on a powerful unix system you can set n_jobs to the number of available threads to speed up the calculation

In [107]:
print("The best parameters are %s with a score of %0.2f" % (grid.best_params_, grid.best_score_))
y_true, y_pred = y_test, grid.predict(X_test)
print(metrics.classification_report(y_true, y_pred))

The best parameters are {'clf__gamma': 0.2, 'vect__max_features': 1500, 'clf__kernel': 'rbf', 'clf__C': 8, 'vect__strip_accents': 'unicode', 'vect__ngram_range': (1, 1), 'vect__stop_words': 'english'} with a score of 0.78
                   precision    recall  f1-score   support

    AaronPressman       1.00      1.00      1.00         9
       AlanCrosby       0.77      0.91      0.83        11
   AlexanderSmith       0.78      0.88      0.82         8
  BenjaminKangLim       0.89      0.89      0.89         9
    BernardHickey       0.73      1.00      0.84         8
      BradDorfman       0.90      0.64      0.75        14
 DarrenSchuettler       0.91      1.00      0.95        10
      DavidLawder       0.62      0.56      0.59         9
    EdnaFernandes       0.86      0.60      0.71        10
      EricAuchard       0.60      0.33      0.43         9
   FumikoFujisaki       1.00      1.00      1.00        10
   GrahamEarnshaw       0.73      0.80      0.76        10
 HeatherSc

## <span>Unsupervised</span><a id='trad_ml_unsupervised'></a> [(to top)](#toc)

### <span>Latent Dirichilet Allocation (LDA)</span><a id='trad_ml_unsupervised_lda'></a> [(to top)](#toc)

In [108]:
from sklearn.decomposition import LatentDirichletAllocation

Vectorizer (using countvectorizer for the sake of example)

In [147]:
vectorizer = CountVectorizer(strip_accents='unicode',
                             lowercase = True,
                            max_features = 1500,
                            stop_words='english', max_df=0.8)
tf_large = vectorizer.fit_transform(clean_paragraphs)

Run the LDA model

In [136]:
n_topics = 10
n_top_words = 25

In [148]:
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=10,
                                learning_method='online',
                                n_jobs=1)
lda_fitted = lda.fit_transform(tf_large)

Visualize top words

In [149]:
def save_top_words(model, feature_names, n_top_words):
    out_list = []
    for topic_idx, topic in enumerate(model.components_):
        out_list.append((topic_idx+1, " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])))
    out_df = pd.DataFrame(out_list, columns=['topic_id', 'top_words'])
    return out_df

In [150]:
result_df = save_top_words(lda, vectorizer.get_feature_names(), n_top_words)

In [151]:
result_df

,topic_id,top_words
0,1,company share million percent group pound mark...
1,2,bank financial market banking fund debt firm n...
2,3,china kong hong chinese beijing people year de...
3,4,thomson ford car television gm new csf cable t...
4,5,government trade percent year czech oil crown ...
5,6,gold bre canada toronto busang north canadian ...
6,7,company service new corp computer internet net...
7,8,tonne market price year trader percent stock w...
8,9,percent year million profit sale analyst share...
9,10,tobacco air conrail boeing state airline compa...


### <span>pyLDAvis</span><a id='trad_ml_unsupervised_pyLDAvis'></a> [(to top)](#toc)

In [121]:
%matplotlib inline
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [152]:
pyLDAvis.sklearn.prepare(lda, tf_large, vectorizer, n_jobs=1)

/home/u1238329/anaconda3/lib/python3.5/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
8      18.742531        1       1 -0.199457 -0.028758
0      18.628928        1       2 -0.072734  0.048877
2      14.374059        1       3  0.090754 -0.027557
6      11.442473        1       4 -0.074860  0.088662
4       9.036709        1       5 -0.024172 -0.020448
1       8.600028        1       6 -0.050096  0.016569
7       6.823840        1       7 -0.144744 -0.134166
3       4.581432        1       8  0.162894  0.173882
9       4.477801        1       9  0.091465  0.112647
5       3.292198        1      10  0.220951 -0.229708, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
183   Default  2812.000000        bank  2812.000000  30.0000  30.0000
287   Default  3599.000000       china  3599.000000  29.0000  29.0000
766   Default  2234.000000        kong  2234.000000  28.0000  28.0000
673   Default  2225.000000        hong  2225.000000  27.0000  27.0000
990   Default  5561.000000     percent  5561.000000  26.0000  26.0000
196   Default  1400.000000     beijing  1400.000000  25.0000  25.0000
1384  Default   975.000000       tonne   975.000000  24.0000  24.0000
844   Default  4237.000000      market  4237.000000  23.0000  23.0000
288   Default  1647.000000     chinese  1647.000000  22.0000  22.0000
1065  Default  1929.000000      profit  1929.000000  21.0000  21.0000
1049  Default  1997.000000       price  1997.000000  20.0000  20.0000
630   Default   631.000000        gold   631.000000  19.0000  19.0000
331   Default  5385.000000     company  5385.000000  18.0000  18.0000
219   Default   613.000000         bre   613.000000  17.0000  17.0000
1238  Default  3390.000000       share  3390.000000  16.0000  16.0000
1200  Default  2140.000000        sale  2140.000000  15.0000  15.0000
937   Default  1708.000000    official  1708.000000  14.0000  14.0000
1097  Default  1445.000000     quarter  1445.000000  13.0000  13.0000
340   Default   809.000000    computer   809.000000  12.0000  12.0000
873   Default  5087.000000     million  5087.000000  11.0000  11.0000
248   Default   550.000000      canada   550.000000  10.0000  10.0000
369   Default  1619.000000        corp  1619.000000   9.0000   9.0000
1230  Default  1552.000000     service  1552.000000   8.0000   8.0000
1373  Default   512.000000     thomson   512.000000   7.0000   7.0000
632   Default  1972.000000  government  1972.000000   6.0000   6.0000
924   Default   617.000000       north   617.000000   5.0000   5.0000
426   Default   785.000000        deng   785.000000   4.0000   4.0000
575   Default  1258.000000   financial  1258.000000   3.0000   3.0000
727   Default   682.000000    internet   682.000000   2.0000   2.0000
593   Default   473.000000        ford   473.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
956   Topic10   108.866842       ounce   109.798892   3.4051  -5.0248
1317  Topic10    99.827439  strathcona   100.780797   3.4041  -5.1115
708   Topic10   162.751592  indonesian   164.820426   3.4010  -4.6227
1449  Topic10   103.449425       walsh   105.286283   3.3960  -5.0758
874   Topic10   272.837218      mining   278.069745   3.3946  -4.1060
1224  Topic10   157.897986       seoul   163.420040   3.3792  -4.6529
1491  Topic10   127.098859    xinjiang   135.428405   3.3501  -4.8699
917   Topic10   143.459464      nickel   154.141461   3.3418  -4.7488
768   Topic10   201.877955      korean   219.872150   3.3282  -4.4072
249   Topic10   286.983285    canadian   335.951200   3.2561  -4.0555
495   Topic10    70.849071     embassy    83.412283   3.2504  -5.4543
1179  Topic10    92.604875        riot   109.751512   3.2437  -5.1866
707   Topic10   145.964191   indonesia   174.319849   3.2361  -4.7315
248   Topic10   448.268246      canada   550.090330   3.2089  -3.6095
767   Topic10   195

## <span style="text-decoration: underline;">Neural Networks</span><a id='nn_ml'></a> [(to top)](#toc)

Interested? Check out the Stanford course CS224n ([Syllabus](http://web.stanford.edu/class/cs224n/syllabus.html))!   